In [28]:
import pandas as pd 
import numpy as np 

In [29]:
df=pd.read_csv("stock_data.csv")

In [30]:
df.head()

,Open,Close,High,Low,Volume,RSI,MACD,Bollinger_Upper,Bollinger_Lower,Sentiment_Score,GDP_Growth,Inflation_Rate,Target
0,0.374639,0.374780,0.373510,0.378390,0.298909,0.847286,0.741715,0.367146,0.366420,0.877177,0.580868,0.038604,0
1,0.950982,0.937746,0.938422,0.946158,0.094805,0.494543,0.881343,0.938396,0.935640,0.907192,0.527044,0.108908,0
2,0.732198,0.719825,0.723644,0.723158,0.126348,0.195471,0.463179,0.710666,0.702300,0.378363,0.351052,0.432540,0
3,0.598823,0.599865,0.596973,0.605322,0.180662,0.736684,0.289076,0.593793,0.586936,0.231614,0.493274,0.946349,0
4,0.156053,0.163410,0.155891,0.166084,0.203646,0.418698,0.318761,0.164158,0.156355,0.191642,0.365116,0.074867,0


In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0    Open            10000 non-null  float64
 1   Close            10000 non-null  float64
 2   High             10000 non-null  float64
 3   Low              10000 non-null  float64
 4   Volume           10000 non-null  float64
 5   RSI              10000 non-null  float64
 6   MACD             10000 non-null  float64
 7   Bollinger_Upper  10000 non-null  float64
 8   Bollinger_Lower  10000 non-null  float64
 9   Sentiment_Score  10000 non-null  float64
 10  GDP_Growth       10000 non-null  float64
 11  Inflation_Rate   10000 non-null  float64
 12  Target           10000 non-null  int64  
dtypes: float64(12), int64(1)
memory usage: 1015.8 KB


In [32]:
df.describe()

,Open,Close,High,Low,Volume,RSI,MACD,Bollinger_Upper,Bollinger_Lower,Sentiment_Score,GDP_Growth,Inflation_Rate,Target
count,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.00000
mean,0.494293,0.495067,0.493072,0.497314,0.496869,0.503171,0.501809,0.496134,0.492211,0.495362,0.500334,0.501918,0.05950
std,0.287715,0.281587,0.284019,0.283283,0.289297,0.288361,0.287825,0.276432,0.276758,0.287750,0.288366,0.290510,0.23657
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
25%,0.246390,0.251759,0.247605,0.252727,0.244266,0.256966,0.253762,0.256406,0.253212,0.244864,0.246618,0.249640,0.00000
50%,0.492662,0.492477,0.491295,0.495413,0.493722,0.506117,0.504232,0.493400,0.489197,0.496597,0.503139,0.502395,0.00000
75%,0.740212,0.736107,0.736170,0.739736,0.750494,0.753489,0.752636,0.732336,0.729294,0.742288,0.750093,0.756607,0.00000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000


In [33]:
df.isnull().sum()

 Open              0
Close              0
High               0
Low                0
Volume             0
RSI                0
MACD               0
Bollinger_Upper    0
Bollinger_Lower    0
Sentiment_Score    0
GDP_Growth         0
Inflation_Rate     0
Target             0
dtype: int64

In [21]:
from sklearn.preprocessing import MinMaxScaler
import warnings
warnings.filterwarnings('ignore')

def load_and_prepare_data(filepath):
    """Load and initial preparation of stock data"""
    df = pd.read_csv(filepath, sep='\t')
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df = df.sort_values('timestamp').reset_index(drop=True)
    df = df[['timestamp', 'open', 'high', 'low', 'close', 'volume']]
    for col in ['open', 'high', 'low', 'close', 'volume']:
        df[col] = pd.to_numeric(df[col], errors='coerce')
    df = df.dropna()
    return df

def calculate_technical_indicators(df):
    """Calculate technical indicators using ONLY past data"""
    df = df.copy()
    df['returns'] = df['close'].pct_change()
    df['log_returns'] = np.log(df['close'] / df['close'].shift(1))
    df['price_range'] = df['high'] - df['low']
    df['price_range_pct'] = (df['high'] - df['low']) / df['close']
    df['gap'] = df['open'] - df['close'].shift(1)
    df['gap_pct'] = (df['open'] - df['close'].shift(1)) / df['close'].shift(1)

    # Simplify moving averages for small data
    df['sma_5'] = df['close'].rolling(window=5, min_periods=1).mean()
    df['sma_10'] = df['close'].rolling(window=10, min_periods=1).mean()
    df['sma_20'] = df['close'].rolling(window=20, min_periods=1).mean()
    df['ema_5'] = df['close'].ewm(span=5, adjust=False).mean()
    df['ema_10'] = df['close'].ewm(span=10, adjust=False).mean()
    df['ema_20'] = df['close'].ewm(span=20, adjust=False).mean()
    df['price_to_sma20'] = (df['close'] - df['sma_20']) / df['sma_20']

    # RSI
    def calculate_rsi(data, period=14):
        delta = data.diff()
        gain = (delta.where(delta > 0, 0)).rolling(window=period, min_periods=1).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(window=period, min_periods=1).mean()
        loss = loss.replace(0, 1e-9)  # prevent division by zero
        rs = gain / loss
        rsi = 100 - (100 / (1 + rs))
        return rsi

    df['rsi_14'] = calculate_rsi(df['close'], 14)
    df['rsi_7'] = calculate_rsi(df['close'], 7)

    # MACD
    df['ema_12'] = df['close'].ewm(span=12, adjust=False).mean()
    df['ema_26'] = df['close'].ewm(span=26, adjust=False).mean()
    df['macd'] = df['ema_12'] - df['ema_26']
    df['macd_signal'] = df['macd'].ewm(span=9, adjust=False).mean()
    df['macd_diff'] = df['macd'] - df['macd_signal']

    # ROC and Momentum (shorter windows)
    df['roc_3'] = ((df['close'] - df['close'].shift(3)) / df['close'].shift(3)) * 100
    df['momentum_3'] = df['close'] - df['close'].shift(3)

    # Volatility
    df['volatility_5'] = df['returns'].rolling(window=5, min_periods=1).std()
    df['volatility_10'] = df['returns'].rolling(window=10, min_periods=1).std()

    # Bollinger Bands
    df['bb_middle'] = df['close'].rolling(window=10, min_periods=1).mean()
    bb_std = df['close'].rolling(window=10, min_periods=1).std()
    df['bb_upper'] = df['bb_middle'] + (bb_std * 2)
    df['bb_lower'] = df['bb_middle'] - (bb_std * 2)
    df['bb_width'] = df['bb_upper'] - df['bb_lower']
    df['bb_position'] = (df['close'] - df['bb_lower']) / (df['bb_upper'] - df['bb_lower'])

    # ATR
    high_low = df['high'] - df['low']
    high_close = np.abs(df['high'] - df['close'].shift())
    low_close = np.abs(df['low'] - df['close'].shift())
    ranges = pd.concat([high_low, high_close, low_close], axis=1)
    true_range = np.max(ranges, axis=1)
    df['atr_7'] = true_range.rolling(window=7, min_periods=1).mean()

    # Volume-based
    df['volume_sma_5'] = df['volume'].rolling(window=5, min_periods=1).mean()
    df['volume_ratio'] = df['volume'] / (df['volume_sma_5'] + 1e-9)
    df['obv'] = (np.sign(df['returns']) * df['volume']).fillna(0).cumsum()

    # Lag features (reduce to 1–3 lags)
    for lag in [1, 2, 3]:
        df[f'close_lag_{lag}'] = df['close'].shift(lag)
        df[f'returns_lag_{lag}'] = df['returns'].shift(lag)
        df[f'volume_lag_{lag}'] = df['volume'].shift(lag)

    # Time features
    df['day_of_week'] = df['timestamp'].dt.dayofweek
    df['month'] = df['timestamp'].dt.month

    # Target
    df['target_price'] = df['close'].shift(-1)
    df['target_direction'] = (df['target_price'] > df['close']).astype(int)
    df['target_return'] = (df['target_price'] - df['close']) / df['close']

    return df

def create_sequences(df, lookback=10, features_to_use=None):
    """Create sequences for LSTM model"""
    if features_to_use is None:
        exclude_cols = ['timestamp', 'target_price', 'target_direction', 'target_return']
        features_to_use = [col for col in df.columns if col not in exclude_cols]

    df_clean = df.dropna()
    X = df_clean[features_to_use].values
    y_price = df_clean['target_price'].values
    y_direction = df_clean['target_direction'].values
    timestamps = df_clean['timestamp'].values

    scaler = MinMaxScaler()
    X_scaled = scaler.fit_transform(X)

    X_sequences, y_prices, y_directions, sequence_timestamps = [], [], [], []
    for i in range(lookback, len(X_scaled)):
        X_sequences.append(X_scaled[i-lookback:i])
        y_prices.append(y_price[i])
        y_directions.append(y_direction[i])
        sequence_timestamps.append(timestamps[i])

    X_sequences = np.array(X_sequences)
    y_prices = np.array(y_prices)
    y_directions = np.array(y_directions)

    return X_sequences, y_prices, y_directions, sequence_timestamps, scaler, features_to_use

def split_time_series(X, y_price, y_direction, timestamps, train_ratio=0.7, val_ratio=0.15):
    """Split data chronologically"""
    n = len(X)
    train_size = int(n * train_ratio)
    val_size = int(n * val_ratio)

    X_train = X[:train_size]
    y_price_train = y_price[:train_size]
    y_dir_train = y_direction[:train_size]
    time_train = timestamps[:train_size]

    X_val = X[train_size:train_size+val_size]
    y_price_val = y_price[train_size:train_size+val_size]
    y_dir_val = y_direction[train_size:train_size+val_size]
    time_val = timestamps[train_size:train_size+val_size]

    X_test = X[train_size+val_size:]
    y_price_test = y_price[train_size+val_size:]
    y_dir_test = y_direction[train_size+val_size:]
    time_test = timestamps[train_size+val_size:]

    print(f"Train set: {len(X_train)} samples")
    print(f"Val set: {len(X_val)} samples")
    print(f"Test set: {len(X_test)} samples")

    return (X_train, y_price_train, y_dir_train, time_train,
            X_val, y_price_val, y_dir_val, time_val,
            X_test, y_price_test, y_dir_test, time_test)


# ============ MAIN EXECUTION ============
if __name__ == "__main__":
    print("Loading data...")
    df = load_and_prepare_data(r"C:\Users\user\OneDrive\Desktop\ML\InterIIT\my-pathway-project\Task 2\Bitcoin_historical_data.csv.csv")
    print(f"Loaded {len(df)} rows")
    print(f"Date range: {df['timestamp'].min()} to {df['timestamp'].max()}")

    print("\nCalculating technical indicators...")
    df = calculate_technical_indicators(df)
    print(f"Created {len(df.columns)} features")

    print("\nSample of engineered features:")
    print(df[['timestamp', 'close', 'returns', 'rsi_14', 'macd', 'sma_10', 'target_direction']].tail())

    print("\nCreating sequences for LSTM...")
    lookback = 10
    X_seq, y_prices, y_directions, seq_times, scaler, feature_names = create_sequences(df, lookback=lookback)

    print(f"Sequence shape: {X_seq.shape}")
    print(f"Number of features: {len(feature_names)}")

    print("\nSplitting data chronologically...")
    splits = split_time_series(X_seq, y_prices, y_directions, seq_times)
    (X_train, y_price_train, y_dir_train, time_train,
     X_val, y_price_val, y_dir_val, time_val,
     X_test, y_price_test, y_dir_test, time_test) = splits

    print("\nSaving processed data...")
    np.savez('processed_data.npz',
             X_train=X_train, y_price_train=y_price_train, y_dir_train=y_dir_train,
             X_val=X_val, y_price_val=y_price_val, y_dir_val=y_dir_val,
             X_test=X_test, y_price_test=y_price_test, y_dir_test=y_dir_test,
             time_test=time_test, feature_names=feature_names)

    print("\n✓ Feature engineering complete!")
    print(f"✓ Ready for model training with {X_train.shape[2]} features")


Loading data...
Loaded 398 rows
Date range: 2024-09-19 23:59:59.999000+00:00 to 2025-10-21 23:59:59.999000+00:00

Calculating technical indicators...
Created 53 features

Sample of engineered features:
                           timestamp        close   returns     rsi_14  \
393 2025-10-17 23:59:59.999000+00:00  106467.7852 -0.015882  26.422675   
394 2025-10-18 23:59:59.999000+00:00  107198.2669  0.006861  27.656860   
395 2025-10-19 23:59:59.999000+00:00  108666.7115  0.013698  28.455538   
396 2025-10-20 23:59:59.999000+00:00  110588.9302  0.017689  29.846478   
397 2025-10-21 23:59:59.999000+00:00  108476.8870 -0.019098  30.891852   

            macd        sma_10  target_direction  
393 -1495.468195  113807.91947                 1  
394 -1867.796516  112192.25913                 1  
395 -2021.080429  110888.37205                 1  
396 -1964.803024  110625.82847                 0  
397 -2066.802201  110392.72924                 0  

Creating sequences for LSTM...
Sequence shape:

In [22]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import numpy as np

# =================== PART 1: RANDOM FOREST ===================

# Flatten sequences for RF (LSTM sequences are 3D: samples x lookback x features)
X_train_rf = X_train.reshape(X_train.shape[0], -1)
X_val_rf = X_val.reshape(X_val.shape[0], -1)
X_test_rf = X_test.reshape(X_test.shape[0], -1)

y_train_rf = y_dir_train
y_val_rf = y_dir_val
y_test_rf = y_dir_test

# Initialize model
rf_model = RandomForestClassifier(
    n_estimators=200,
    max_depth=5,
    random_state=42,
    n_jobs=-1
)

# ============ 1. Train on initial training set ============
rf_model.fit(X_train_rf, y_train_rf)

# ============ 2. Evaluate on validation set ============
y_val_pred = rf_model.predict(X_val_rf)
val_acc = accuracy_score(y_val_rf, y_val_pred)
print(f"Validation Accuracy: {val_acc:.4f}")

# ============ 3. Time-series safe rolling prediction on test set ============
y_pred_rolling = []

# Start with training set
X_train_rf_full = X_train_rf.copy()
y_train_full = y_train_rf.copy()

for i in range(X_test_rf.shape[0]):
    # Train on all past data up to current test point
    rf_model = RandomForestClassifier(
        n_estimators=200,
        max_depth=5,
        random_state=42,
        n_jobs=-1
    )
    rf_model.fit(X_train_rf_full, y_train_full)
    
    # Predict the current test sample
    x_test_sample = X_test_rf[i].reshape(1, -1)
    y_pred = rf_model.predict(x_test_sample)
    y_pred_rolling.append(y_pred[0])
    
    # Include this test point into training for next iteration
    X_train_rf_full = np.vstack([X_train_rf_full, x_test_sample])
    y_train_full = np.append(y_train_full, y_dir_test[i])

# ============ 4. Evaluate test set ============
test_acc = accuracy_score(y_dir_test, y_pred_rolling)
print(f"Time-series safe Test Accuracy: {test_acc:.4f}")
print("\nConfusion Matrix:")
print(confusion_matrix(y_dir_test, y_pred_rolling))
print("\nClassification Report:")
print(classification_report(y_dir_test, y_pred_rolling))


Validation Accuracy: 0.5263
Time-series safe Test Accuracy: 0.4138

Confusion Matrix:
[[13 14]
 [20 11]]

Classification Report:
              precision    recall  f1-score   support

           0       0.39      0.48      0.43        27
           1       0.44      0.35      0.39        31

    accuracy                           0.41        58
   macro avg       0.42      0.42      0.41        58
weighted avg       0.42      0.41      0.41        58



In [23]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

def time_series_evaluation_rf(X, y, timestamps, n_splits=5):
    """
    Perform time-series aware evaluation using expanding window.
    Ensures no future data is used for training.
    
    Parameters:
    - X: Features (2D array or 3D for sequences flattened)
    - y: Target (binary or multi-class)
    - timestamps: Corresponding timestamps
    - n_splits: Number of splits for evaluation
    
    Returns:
    - list of metrics for each split
    """
    n_samples = len(X)
    split_size = n_samples // (n_splits + 1)
    results = []

    for i in range(1, n_splits + 1):
        train_end = split_size * i
        test_end = split_size * (i + 1)

        # Ensure test_end does not exceed length
        if test_end > n_samples:
            test_end = n_samples

        X_train, X_test = X[:train_end], X[train_end:test_end]
        y_train, y_test = y[:train_end], y[train_end:test_end]
        
        # Flatten sequences if needed
        if len(X_train.shape) == 3:
            X_train_flat = X_train.reshape(X_train.shape[0], -1)
            X_test_flat = X_test.reshape(X_test.shape[0], -1)
        else:
            X_train_flat, X_test_flat = X_train, X_test

        # Train Random Forest
        rf = RandomForestClassifier(n_estimators=200, random_state=42)
        rf.fit(X_train_flat, y_train)
        y_pred = rf.predict(X_test_flat)

        # Metrics
        acc = accuracy_score(y_test, y_pred)
        report = classification_report(y_test, y_pred, output_dict=True)
        cm = confusion_matrix(y_test, y_pred)

        print(f"\nSplit {i} | Train samples: {len(X_train)} | Test samples: {len(X_test)}")
        print(f"Accuracy: {acc:.4f}")
        print("Confusion Matrix:\n", cm)

        results.append({
            'split': i,
            'train_size': len(X_train),
            'test_size': len(X_test),
            'accuracy': acc,
            'report': report,
            'confusion_matrix': cm
        })

    return results

# ================= MAIN EXECUTION =================

if __name__ == "__main__":
    # Load your processed sequences and targets from Step 1
    data = np.load('processed_data.npz', allow_pickle=True)
    X_train = data['X_train']
    y_dir_train = data['y_dir_train']
    X_val = data['X_val']
    y_dir_val = data['y_dir_val']
    X_test = data['X_test']
    y_dir_test = data['y_dir_test']
    time_test = data['time_test']

    # Combine train + val for time-series evaluation
    X_all = np.concatenate([X_train, X_val, X_test], axis=0)
    y_all = np.concatenate([y_dir_train, y_dir_val, y_dir_test], axis=0)
    timestamps_all = np.concatenate([data['X_train'], data['X_val'], data['X_test']], axis=0)

    print("\nStarting Time-Series Evaluation for Random Forest...")
    results = time_series_evaluation_rf(X_all, y_all, timestamps_all, n_splits=5)



Starting Time-Series Evaluation for Random Forest...

Split 1 | Train samples: 63 | Test samples: 63
Accuracy: 0.5079
Confusion Matrix:
 [[15 17]
 [14 17]]

Split 2 | Train samples: 126 | Test samples: 63
Accuracy: 0.5238
Confusion Matrix:
 [[10 25]
 [ 5 23]]

Split 3 | Train samples: 189 | Test samples: 63
Accuracy: 0.5397
Confusion Matrix:
 [[21  6]
 [23 13]]

Split 4 | Train samples: 252 | Test samples: 63
Accuracy: 0.5079
Confusion Matrix:
 [[15 15]
 [16 17]]

Split 5 | Train samples: 315 | Test samples: 63
Accuracy: 0.5079
Confusion Matrix:
 [[20 12]
 [19 12]]


In [24]:
from sklearn.ensemble import RandomForestClassifier
import numpy as np

# Step 1: Fit Random Forest on flattened sequences
rf = RandomForestClassifier(n_estimators=500, random_state=42)
X_train_flat = X_train.reshape(X_train.shape[0], -1)  # flatten time dimension
rf.fit(X_train_flat, y_dir_train)

# Step 2: Get feature importances
importances = rf.feature_importances_

# Flattened feature names corresponding to X_train_flat
feature_names_flat = []
lookback = X_train.shape[1]
for f in feature_names:  # feature_names from your LSTM feature list
    feature_names_flat.extend([f + f"_t-{i}" for i in range(lookback, 0, -1)])

# Create a sorted list of features by importance
feat_imp_df = pd.DataFrame({
    'feature': feature_names_flat,
    'importance': importances
}).sort_values(by='importance', ascending=False)

# Step 3: Select top N features (e.g., top 20)
top_features_flat = feat_imp_df['feature'].head(20).tolist()

# Map back to original base features for LSTM input
top_base_features = list({f.split("_t-")[0] for f in top_features_flat})
print("Selected features for LSTM:", top_base_features)

# Step 4: Re-create sequences using only top features
X_seq_top, y_prices_top, y_directions_top, seq_times_top, scaler_top, feature_names_top = create_sequences(
    df, lookback=X_train.shape[1], features_to_use=top_base_features
)

# Step 5: Split again chronologically
splits_top = split_time_series(X_seq_top, y_prices_top, y_directions_top, seq_times_top)
(X_train_top, y_price_train_top, y_dir_train_top, time_train_top,
 X_val_top, y_price_val_top, y_dir_val_top, time_val_top,
 X_test_top, y_price_test_top, y_dir_test_top, time_test_top) = splits_top

print(f"Reduced feature set ready for LSTM: {len(top_base_features)} features")


Selected features for LSTM: ['close_lag_1', 'ema_5', 'sma_10', 'macd_signal', 'bb_lower', 'rsi_14', 'roc_3', 'close_lag_3', 'ema_26', 'bb_position', 'volume_ratio', 'close', 'gap', 'month', 'ema_10', 'day_of_week', 'bb_middle', 'volume_lag_2']
Train set: 268 samples
Val set: 57 samples
Test set: 58 samples
Reduced feature set ready for LSTM: 18 features


In [26]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np

# ==========================
# CONFIGURATION
# ==========================
lookback = 20  # strictly past 20 rows
n_features = X_train_top.shape[2]  # number of features
batch_size = 8  # smaller batch for tiny data

# ==========================
# MODEL
# ==========================

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense, Dropout

lookback = X_train_top.shape[1]
n_features = X_train_top.shape[2]

model = Sequential([
    GRU(16, return_sequences=True, input_shape=(lookback, n_features)),
    Dropout(0.2),
    GRU(8),
    Dense(8, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

early_stop = EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True,
    verbose=1
)

history = model.fit(
    X_train_top, y_dir_train_top,
    validation_data=(X_val_top, y_dir_val_top),
    epochs=100,
    batch_size=8,
    callbacks=[early_stop],
    verbose=1,
    shuffle=False
)


Epoch 1/100
34/34 ━━━━━━━━━━━━━━━━━━━━ 8s 31ms/step - accuracy: 0.5149 - loss: 0.6938 - val_accuracy: 0.5263 - val_loss: 0.6878
Epoch 2/100
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.5000 - loss: 0.6937 - val_accuracy: 0.5614 - val_loss: 0.6876
Epoch 3/100
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.4963 - loss: 0.6949 - val_accuracy: 0.5614 - val_loss: 0.6876
Epoch 4/100
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.4701 - loss: 0.6998 - val_accuracy: 0.6140 - val_loss: 0.6890
Epoch 5/100
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.5299 - loss: 0.6930 - val_accuracy: 0.5263 - val_loss: 0.6878
Epoch 6/100
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.5373 - loss: 0.6934 - val_accuracy: 0.5263 - val_loss: 0.6880
Epoch 7/100
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.5522 - loss: 0.6882 - val_accuracy: 0.5263 - val_loss: 0.6881
Epoch 8/100
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.5224 - loss: 0.6905 - val_accuracy: 0.

In [27]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Dense, Dropout, BatchNormalization, GlobalMaxPooling1D
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import confusion_matrix, classification_report

# ==========================
# CONFIGURATION
# ==========================
lookback = X_train_top.shape[1]  # sequence length
n_features = X_train_top.shape[2]  # number of features

# ==========================
# MODEL
# ==========================
from tensorflow.keras.layers import Input, Conv1D, Dense, Dropout, BatchNormalization, GlobalAveragePooling1D, MultiHeadAttention
from tensorflow.keras.models import Model

inputs = Input(shape=(lookback, n_features))

x = Conv1D(16, kernel_size=3, activation='relu', padding='causal')(inputs)
x = BatchNormalization()(x)
x = Dropout(0.2)(x)

# tiny attention
attn_output = MultiHeadAttention(num_heads=2, key_dim=8)(x, x)
x = x + attn_output  # residual
x = GlobalAveragePooling1D()(x)

x = Dense(16, activation='relu')(x)
x = Dropout(0.2)(x)
outputs = Dense(1, activation='sigmoid')(x)

model = Model(inputs, outputs)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()


# ==========================
# CALLBACKS
# ==========================
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True,
    verbose=1
)

# ==========================
# TRAINING
# ==========================
history = model.fit(
    X_train_top, y_dir_train_top,
    validation_data=(X_val_top, y_dir_val_top),
    epochs=100,
    batch_size=16,
    callbacks=[early_stop],
    verbose=1
)

# ==========================
# EVALUATION
# ==========================
test_loss, test_acc = model.evaluate(X_test_top, y_dir_test_top, verbose=0)
print(f"\nTest Accuracy: {test_acc:.4f}")

# ==========================
# PREDICTIONS
# ==========================
y_pred_prob = model.predict(X_test_top)
y_pred_dir = (y_pred_prob > 0.5).astype(int)

# ==========================
# CONFUSION MATRIX & REPORT
# ==========================
cm = confusion_matrix(y_dir_test_top, y_pred_dir)
print("Confusion Matrix:\n", cm)

report = classification_report(y_dir_test_top, y_pred_dir, digits=4)
print("\nClassification Report:\n", report)


Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2       │ (None, 10, 18)    │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d (Conv1D)     │ (None, 10, 16)    │        880 │ input_layer_2[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 10, 16)    │         64 │ conv1d[0][0]      │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_2 (Dropout) │ (None, 10, 16)    │          0 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 10, 16)    │      1,088 │ dropout_2[0][0],  │
│ (MultiHeadAttentio… │                   │            │ dropout_2[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 10, 16)    │          0 │ dropout_2[0][0],  │
│                     │                   │            │ multi_head_atten… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 16)        │          0 │ add[0][0]         │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_4 (Dense)     │ (None, 16)        │        272 │ global_average_p… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_4 (Dropout) │ (None, 16)        │          0 │ dense_4[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_5 (Dense)     │ (None, 1)         │         17 │ dropout_4[0][0]   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 2,321 (9.07 KB)

 Trainable params: 2,289 (8.94 KB)

 Non-trainable params: 32 (128.00 B)

Epoch 1/100
17/17 ━━━━━━━━━━━━━━━━━━━━ 5s 36ms/step - accuracy: 0.4888 - loss: 0.7168 - val_accuracy: 0.4737 - val_loss: 0.7032
Epoch 2/100
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.5299 - loss: 0.6982 - val_accuracy: 0.4386 - val_loss: 0.7020
Epoch 3/100
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.5075 - loss: 0.6966 - val_accuracy: 0.4035 - val_loss: 0.6991
Epoch 4/100
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.5896 - loss: 0.6758 - val_accuracy: 0.4211 - val_loss: 0.6977
Epoch 5/100
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.5336 - loss: 0.6868 - val_accuracy: 0.4561 - val_loss: 0.6956
Epoch 6/100
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.5448 - loss: 0.6793 - val_accuracy: 0.4561 - val_loss: 0.6977
Epoch 7/100
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.5709 - loss: 0.6688 - val_accuracy: 0.5263 - val_loss: 0.6965
Epoch 8/100
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.5597 - loss: 0.6842 - val_accuracy: 0.